# PE-GNN - Example notebook

This notebook includes the *PE-GNN* method proposed in the paper [" 	Positional Encoder Graph Neural Networks for Geographic Data "](https://arxiv.org/abs/2111.10144), implemented in `PyTorch`. It is optimized for running on [Google Colab](colab.research.google.com/).

## Install Dependencies
Our experiments rely on *PyG* / *PyTorch Geometric* for the GNN backbones. See here for more details see: https://github.com/pyg-team/pytorch_geometric 

In [ ]:
# Check version of pytorch and cuda -- following assumes PyTorch version 1.10.0 and cuda 11.1
! python -c "import torch; print(torch.__version__)"
! python -c "import torch; print(torch.version.cuda)"

In [5]:
! pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric geomloss

  Using cached torch_scatter-2.0.9-cp39-cp39-macosx_10_9_x86_64.whl
  Using cached torch_sparse-0.6.15.tar.gz (2.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached torch_cluster-1.6.0.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_spline_conv-1.2.1.tar.gz (13 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.15-cp39-cp39-macosx_10_9_x86_64.whl size=549980 sha256=27b9074f69cdfdab614410ea0cdf737671b8adb14d5d0b96ce96cf2188df1799
  Stored in directory: /Users/fnachtigall/Library/Caches/pip/wheels/bc/9e/f5/f493101401d551fa7223cbdfc220c26772c697c0f5ebeccca8
anceled
ERROR: Operation cancelled by user


In [3]:
# # Install pytorch geometric -- make sure torch and cuda numbers at the end are consistent with your versoin
! pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
! pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
! pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
! pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
! pip install torch-geometric
! pip install geomloss[full]

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
ERROR: Could not find a version that satisfies the requirement torch-scatter (from versions: none)
ERROR: No matching distribution found for torch-scatter
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
ERROR: Could not find a version that satisfies the requirement torch-sparse (from versions: none)
ERROR: No matching distribution found for torch-sparse
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
ERROR: Could not find a version that satisfies the requirement torch-cluster (from versions: none)
ERROR: No matching distribution found for torch-cluster
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
ERROR: Could not find a version that satisfies the requirement torch-spline-conv (from versions: none)
ERROR: No matching distribution found for torch-spline-conv
zsh:1: no matches found: geomloss[full]


Import packages

In [2]:
# Imports
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import Sampler
import random
import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.nn import GCNConv, knn_graph
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error
from geopy.distance import distance
import math
import matplotlib.pyplot as plt

import os
import datetime
import sys
import requests
from urllib.request import urlretrieve
import urllib.request, json 
import zipfile
import subprocess

import torch
# import torchvision.transforms as transforms
# import torchvision
import torch.nn as nn
import torch.nn.functional as F

from decimal import Decimal, getcontext
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset, IterableDataset, DataLoader

import numpy as np
import geopandas as gpd
import pandas as pd
import scipy
from scipy import sparse

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import normalize
from sklearn import metrics

import torch
import argparse
import glob
import os
import time
import tqdm

from datetime import datetime
import numpy as np
from urllib.request import urlretrieve
import urllib.request, json
from torch.utils.data import Dataset, IterableDataset, DataLoader
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

# Load the TensorBoard notebook extension
# %load_ext tensorboard

/usr/local/Caskroom/miniconda/base/envs/space2vec/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
! pip install tensorboard

## Data utils

Functions to download and process the experimental datasets.

In [4]:
import io
import requests
from urllib import request 
from zipfile import ZipFile
from pathlib import Path
import csv
import numpy as np
import pandas as pd
import sklearn.datasets
from functools import reduce

def normal(x,min_val=0):
  '''
  Normalize a vector

  Parameters:
  x = numerical vector
  min_val = integer; choice of [0,1], setting whether normalization in range[0,1] or [-1,1]

  Return:
  x_norm = normalized vector
  '''
  x_min = np.min(x)
  x_max = np.max(x)
  if x_min == 0 and x_max == 0:
    return x
  if min_val == -1:
    x_norm = 2 * ((x - x_min) / (x_max - x_min)) - 1
  if min_val== 0:
    x_norm = ((x - x_min) / (x_max - x_min))
  return x_norm

def get_election_data(pred="gop_2016",norm_x=True,norm_y=True,norm_min_val=0,spat_int=True):
  '''
  Download and process the Election dataset used in CorrelationGNN (https://arxiv.org/abs/2002.08274)

  Parameters:
  pred = numeric; outcome variable to be returned; choose from ["dem_2016",
                                                       "gop_2016",
                                                       "MedianIncome2016",
                                                       "R_NET_MIG_2016",
                                                       "R_birth_2016",
                                                       "R_death_2016",
                                                       "BachelorRate2016",
                                                       "Unemployment_rate_2016"]
  norm_x = logical; should features be normalized
  norm_y = logical; should outcome be normalized
  norm_min_val = integer; choice of [0,1], setting whether normalization in range[0,1] or [-1,1]

  Return:
  coords = spatial coordinates (lon/lat)
  x = features at location (excluding outcome variable)
  y = outcome variable
  '''

  Path("./election_data").mkdir(parents=True, exist_ok=True)
  zipurl = 'https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2020_Gazetteer/2020_Gaz_counties_national.zip'
  with request.urlopen(zipurl) as zipresp:
      with ZipFile(io.BytesIO(zipresp.read())) as zfile:
          zfile.extractall('./election_data')

  geo = pd.read_csv("./election_data/2020_Gaz_counties_national.txt",sep='\t')
  geo = geo.rename(columns={"GEOID":"FIPS",'INTPTLONG                                                                                                               ':'INTPTLONG'})

  url = 'https://raw.githubusercontent.com/000Justin000/gnn-residual-correlation/master/datasets/election/education.csv'
  url_open = request.urlopen(url)
  edu = pd.read_csv(io.StringIO(url_open.read().decode('utf-8'))) 

  url = 'https://raw.githubusercontent.com/000Justin000/gnn-residual-correlation/master/datasets/election/election.csv'
  url_open = request.urlopen(url)
  ele = pd.read_csv(io.StringIO(url_open.read().decode('utf-8'))) 
  ele = ele.rename(columns={"fips_code":"FIPS"})

  url = 'https://raw.githubusercontent.com/000Justin000/gnn-residual-correlation/master/datasets/election/income.csv'
  url_open = request.urlopen(url)
  inc = pd.read_csv(io.StringIO(url_open.read().decode('utf-8'))) 

  url = 'https://raw.githubusercontent.com/000Justin000/gnn-residual-correlation/master/datasets/election/unemployment.csv'
  url_open = request.urlopen(url)
  une = pd.read_csv(io.StringIO(url_open.read().decode('utf-8'))) 

  url = 'https://raw.githubusercontent.com/000Justin000/gnn-residual-correlation/master/datasets/election/population.csv'
  url_open = request.urlopen(url)
  pop = pd.read_csv(io.StringIO(url_open.read().decode('utf-8'))) 

  dfs = [geo,edu,ele,inc,une,pop]
  data = reduce(lambda  left,right: pd.merge(left,right,on=['FIPS'],how='outer'), dfs)
  data = data.replace({',':''}, regex=True)

  out_data = np.array([data.INTPTLONG,data.INTPTLAT,data.dem_2016,data.gop_2016,data.MedianIncome2016,data.R_NET_MIG_2016,data.R_birth_2016,data.R_death_2016,data.BachelorRate2016,data.Unemployment_rate_2016]).T.astype(float)
  out_data = out_data[~np.isnan(out_data).any(axis=1)]
  out_data = out_data[(out_data[:,0] > -130) & (out_data[:,0] < -50) & (out_data[:,1] > 22) & (out_data[:,1] < 50)]

  coords = out_data[:,:2]
  if pred == "dem_2016":
    y = out_data[:,2]
    x = out_data[:,3:]
  if pred == "gop_2016":
    y = out_data[:,3]
    x = out_data[:,[2,4,5,6,7,8,9]]
  if pred == "MedianIncome2016":
    y = out_data[:,4]
    x = out_data[:,[2,3,5,6,7,8,9]]
  if pred == "R_NET_MIG_2016":
    y = out_data[:,5]
    x = out_data[:,[2,3,4,6,7,8,9]]
  if pred == "R_birth_2016":
    y = out_data[:,6]
    x = out_data[:,[2,3,4,5,7,8,9]]
  if pred == "R_death_2016":
    y = out_data[:,7]
    x = out_data[:,[2,3,4,5,6,8,9]]
  if pred == "BachelorRate2016":
    y = out_data[:,8]
    x = out_data[:,[2,3,4,5,6,7,9]]
  if pred == "Unemployment_rate_2016":
    y = out_data[:,9]
    x = out_data[:,[2,3,4,5,6,7,8]]

  if norm_y==True:
    y = normal(y,norm_min_val)
  if norm_x==True:
    for i in range(x.shape[1]):
      x[:,i] = normal(x[:,i],norm_min_val)
  if spat_int==True:
      x = torch.ones(x.shape[0],1)
  return torch.tensor(coords), torch.tensor(x), torch.tensor(y)

def get_cali_housing_data(norm_x=True,norm_y=True,norm_min_val=0,spat_int=False):
  '''
  Download and process the California Housing Dataset

  Parameters:
  norm_x = logical; should features be normalized
  norm_y = logical; should outcome be normalized
  norm_min_val = integer; choice of [0,1], setting whether normalization in range[0,1] or [-1,1]

  Return:
  coords = spatial coordinates (lon/lat)
  x = features at location
  y = outcome variable
  '''
  cali_housing_ds = sklearn.datasets.fetch_california_housing()
  coords = np.array(cali_housing_ds.data[:,6:])
  y = np.array(cali_housing_ds.target)
  x = np.array(cali_housing_ds.data[:,:6])
  if norm_y==True:
    y = normal(y,norm_min_val)
  if norm_x==True:
    for i in range(x.shape[1]):
      x[:,i] = normal(x[:,i],norm_min_val)
  if spat_int==True:
    x = torch.ones(x.shape[0],1)
  return torch.tensor(coords), torch.tensor(x), torch.tensor(y)

def get_3d_road_data(norm_y=True,norm_min_val=0):
  '''
  Download and process the 3d road dataset

  Parameters:
  norm_x = logical; should features be normalized
  norm_y = logical; should outcome be normalized
  norm_min_val = integer; choice of [0,1], setting whether normalization in range[0,1] or [-1,1]

  Return:
  coords = spatial coordinates (lon/lat)
  x = features at location; empty for this dataset
  y = outcome variable
  '''
  # Both of the above sources contain the 3d road dataset
  #url="https://archive.ics.uci.edu/ml/machine-learning-databases/00246/3D_spatial_network.txt"
  url="http://nrvis.com/data/mldata/3D_spatial_network.csv"
  s=requests.get(url).content
  c=pd.read_csv(io.StringIO(s.decode('utf-8')))
  c.columns = ["id","x","y","z"]
  coords = np.array(c[["x","y"]])
  y = np.array(c[["z"]])
  if norm_y==True:
    y = normal(y,norm_min_val)

  return torch.tensor(coords), None, torch.tensor(y)

def get_air_temp_data(pred="temp",norm_y=True,norm_x=True,norm_min_val=0):
  '''
  Download and process the Global Air Temperature dataset

  Parameters:
  pred = numeric; outcome variable to be returned; choose from ["temp", "prec"]
  norm_y = logical; should outcome be normalized
  norm_min_val = integer; choice of [0,1], setting whether normalization in range[0,1] or [-1,1]

  Return:
  coords = spatial coordinates (lon/lat)
  x = features at location
  y = outcome variable
  '''
  url = 'https://springernature.figshare.com/ndownloader/files/12609182'
  url_open = request.urlopen(url)
  inc = np.array(pd.read_csv(io.StringIO(url_open.read().decode('utf-8'))))
  coords = inc[:,:2]
  if pred=="temp":
    y = inc[:,4].reshape(-1)
    x = inc[:,5]
  else:
    y = inc[:,5].reshape(-1)
    x = inc[:,4]
  if norm_y==True:
    y = normal(y,norm_min_val)
  if norm_x==True:
    x = normal(x,norm_min_val).reshape(-1,1)

  return torch.tensor(coords), torch.tensor(x), torch.tensor(y)

class MyDataset(Dataset):
    def __init__(self, x, y, coords):
      self.features = x
      self.target = y
      self.coords = coords
    def __len__(self):
      return len(self.features)
    def __getitem__(self, idx):
      return torch.tensor(self.features[idx]), torch.tensor(self.target[idx]), torch.tensor(self.coords[idx])

In [18]:
def get_eubucco_data():
    data_dir = os.path.join('../ufo-prediction/demo')

    path_data_NLD = os.path.join(data_dir, 'df-NLD-exp.pkl')
    path_data_FRA = os.path.join(data_dir, 'df-FRA-exp.pkl')
    path_data_ESP = os.path.join(data_dir, 'df-ESP-exp.pkl')
    df = pd.read_pickle(path_data_NLD)
    # df = df[df['city'] == 'Dronten']

    df = df[df['age'] > 1900]
    df = df[df['age'] < 2020]

    coords = df[['lat', 'lon']].values
    x = df[SELECTED_FEATURES].values
    y = df['age'].values

    # y = normal(y, min_val=0)
    for i in range(x.shape[1]):
      x[:,i] = normal(x[:,i], min_val=0)

    return torch.tensor(coords), torch.tensor(x), torch.tensor(y)



In [19]:
import itertools
BUILDING_FEATURES = [
    'FootprintArea',
    'Perimeter',
    'Phi',
    'LongestAxisLength',
    'Elongation',
    'Convexity',
    'Orientation',
    'Corners',
    'CountTouches',
    'SharedWallLength',
    'lat',
    'lon',
]
BUILDING_FEATURES_NEIGHBORHOOD = [
    'av_convexity_within_buffer_100',
    'av_convexity_within_buffer_500',
    'av_elongation_within_buffer_100',
    'av_elongation_within_buffer_500',
    'av_footprint_area_within_buffer_100',
    'av_footprint_area_within_buffer_500',
    'av_orientation_within_buffer_100',
    'av_orientation_within_buffer_500',
    'buildings_within_buffer_100',
    'buildings_within_buffer_500',
    'std_convexity_within_buffer_100',
    'std_convexity_within_buffer_500',
    'std_elongation_within_buffer_100',
    'std_elongation_within_buffer_500',
    'std_footprint_area_within_buffer_100',
    'std_footprint_area_within_buffer_500',
    'std_orientation_within_buffer_100',
    'std_orientation_within_buffer_500',
    'total_ft_area_within_buffer_100',
    'total_ft_area_within_buffer_500',
]
BLOCK_FEATURES = [
    'AvBlockFootprintArea',
    'BlockConvexity',
    'BlockCorners',
    'BlockElongation',
    'BlockLength',
    'BlockLongestAxisLength',
    'BlockOrientation',
    'BlockPerimeter',
    'BlockTotalFootprintArea',
    'StdBlockFootprintArea',
]
BLOCK_FEATURES_NEIGHBORHOOD = [
    'blocks_within_buffer_100',
    'blocks_within_buffer_500',
    'av_block_av_footprint_area_within_buffer_100',
    'av_block_av_footprint_area_within_buffer_500',
    'av_block_footprint_area_within_buffer_100',
    'av_block_footprint_area_within_buffer_500',
    'av_block_length_within_buffer_100',
    'av_block_length_within_buffer_500',
    'av_block_orientation_within_buffer_100',
    'av_block_orientation_within_buffer_500',
    'std_block_av_footprint_area_within_buffer_100',
    'std_block_av_footprint_area_within_buffer_500',
    'std_block_footprint_area_within_buffer_100',
    'std_block_footprint_area_within_buffer_500',
    'std_block_length_within_buffer_100',
    'std_block_length_within_buffer_500',
    'std_block_orientation_within_buffer_100',
    'std_block_orientation_within_buffer_500',
]
SBB_FEATURES = [
    'street_based_block_area',
    'street_based_block_corners',
    'street_based_block_phi',
]
STREET_ONLY_FEATURES = [
    'dist_to_closest_int',
    'distance_to_closest_street',
    'street_length_closest_street',
    'street_openness_closest_street',
    'street_width_av_closest_street',
    'street_width_std_closest_street',
]

STREET_FEATURES = [
    'street_based_block_area',
    'street_based_block_corners',
    'street_based_block_phi',

    'dist_to_closest_int',
    'distance_to_closest_street',
    'street_length_closest_street',
    'street_openness_closest_street',
    'street_width_av_closest_street',
    'street_width_std_closest_street',
]
SBB_FEATURES_NEIGHBORHOOD = [
    'street_based_block_av_area_inter_buffer_100',
    'street_based_block_av_area_inter_buffer_500',
    'street_based_block_av_phi_inter_buffer_100',
    'street_based_block_av_phi_inter_buffer_500',
    'street_based_block_number_inter_buffer_100',
    'street_based_block_number_inter_buffer_500',
    'street_based_block_std_area_inter_buffer_100',
    'street_based_block_std_area_inter_buffer_500',
    'street_based_block_std_orientation_inter_buffer_100',
    'street_based_block_std_orientation_inter_buffer_500',
    'street_based_block_std_phi_inter_buffer_100',
    'street_based_block_std_phi_inter_buffer_500',
]
STREET_ONLY_FEATURES_NEIGHBORHOOD = [
    'intersection_count_within_100',
    'intersection_count_within_500',
    'street_length_av_within_buffer_100',
    'street_length_av_within_buffer_500',
    'street_length_std_within_buffer_100',
    'street_length_std_within_buffer_500',
    'street_length_total_within_buffer_100',
    'street_length_total_within_buffer_500',
    'street_width_av_within_buffer_100',
    'street_width_av_within_buffer_500',
    'street_width_std_within_buffer_100',
    'street_width_std_within_buffer_500'
]
STREET_FEATURES_NEIGHBORHOOD = [
    'street_based_block_av_area_inter_buffer_100',
    'street_based_block_av_area_inter_buffer_500',
    'street_based_block_av_phi_inter_buffer_100',
    'street_based_block_av_phi_inter_buffer_500',
    'street_based_block_number_inter_buffer_100',
    'street_based_block_number_inter_buffer_500',
    'street_based_block_std_area_inter_buffer_100',
    'street_based_block_std_area_inter_buffer_500',
    'street_based_block_std_orientation_inter_buffer_100',
    'street_based_block_std_orientation_inter_buffer_500',
    'street_based_block_std_phi_inter_buffer_100',
    'street_based_block_std_phi_inter_buffer_500',

    'intersection_count_within_100',
    'intersection_count_within_500',
    'street_length_av_within_buffer_100',
    'street_length_av_within_buffer_500',
    'street_length_std_within_buffer_100',
    'street_length_std_within_buffer_500',
    'street_length_total_within_buffer_100',
    'street_length_total_within_buffer_500',
    'street_width_av_within_buffer_100',
    'street_width_av_within_buffer_500',
    'street_width_std_within_buffer_100',
    'street_width_std_within_buffer_500'
]
STREET_FEATURES_CENTRALITY = [
    'street_betweeness_global_av_within_buffer_100',
    'street_betweeness_global_av_within_buffer_500',
    'street_betweeness_global_max_within_buffer_100',
    'street_betweeness_global_max_within_buffer_500',
    'street_betweeness_global_closest_street',
    'street_closeness_global_closest_street',
    'street_closeness_500_closest_street',
    'street_closeness_500_av_within_buffer_100',
    'street_closeness_500_av_within_buffer_500',
    'street_closeness_500_max_within_buffer_100',
    'street_closeness_500_max_within_buffer_500',
]
CITY_FEATURES = [
    'total_buildings_city',
    'n_detached_buildings',
    'total_buildings_footprint_city',
    'av_building_footprint_city',
    'std_building_footprint_city',

    'blocks_2_to_4',
    'blocks_5_to_9',
    'blocks_10_to_19',
    'blocks_20_to_inf',

    'total_length_street_city',
    'av_length_street_city',
    'intersections_count',

    'av_area_block_city',
    'std_area_block_city',
    'total_number_block_city',
]
LANDUSE_FEATURES = [
    'bld_in_lu_agricultural',
    'bld_in_lu_industrial_commercial',
    'bld_in_lu_natural',
    'bld_in_lu_other',
    'bld_in_lu_roads',
    'bld_in_lu_urban_fabric',
    'bld_in_lu_urban_green',
    'bld_in_lu_water',
    'bld_in_lu_ocean_country',
    'bld_in_lu_railways',
    'bld_in_lu_ports_airports',
    'lu_ocean_country_within_buffer_100',
    'lu_natural_within_buffer_100',
    'lu_industrial_commercial_within_buffer_100',
    'lu_other_within_buffer_100',
    'lu_water_within_buffer_100',
    'lu_urban_green_within_buffer_100',
    'lu_agricultural_within_buffer_100',
    'lu_railways_within_buffer_100',
    'lu_urban_fabric_within_buffer_100',
    'lu_ports_airports_within_buffer_100',
    'lu_roads_within_buffer_100',
    'lu_ocean_country_within_buffer_500',
    'lu_natural_within_buffer_500',
    'lu_industrial_commercial_within_buffer_500',
    'lu_other_within_buffer_500',
    'lu_water_within_buffer_500',
    'lu_urban_green_within_buffer_500',
    'lu_agricultural_within_buffer_500',
    'lu_railways_within_buffer_500',
    'lu_urban_fabric_within_buffer_500',
    'lu_ports_airports_within_buffer_500',
    'lu_roads_within_buffer_500'
]
SELECTED_FEATURES = [
    'total_ft_area_within_buffer_100',
    'street_closeness_global_closest_street',
    'StdBlockFootprintArea',
    'street_width_av_closest_street',
    'std_convexity_within_buffer_100',
    'av_convexity_within_buffer_100',
    # 'lon',
    'Phi',
    'street_betweeness_global_closest_street',
    'Convexity',
    'BlockLongestAxisLength',
    'street_width_std_within_buffer_500',
    'street_width_std_closest_street',
    'std_block_av_footprint_area_within_buffer_100',
    'distance_to_closest_street',
    'total_ft_area_within_buffer_500',
    'AvBlockFootprintArea',
    # 'lat',
    'BlockElongation',
    'BlockTotalFootprintArea',
    'street_betweeness_global_av_within_buffer_100',
    'Corners',
    'FootprintArea',
    'std_block_footprint_area_within_buffer_100',
    'buildings_within_buffer_100',
    'buildings_within_buffer_500',
    'av_footprint_area_within_buffer_100',
    'av_elongation_within_buffer_100'
]
FEATURES = list(itertools.chain(
    BUILDING_FEATURES,
    BUILDING_FEATURES_NEIGHBORHOOD,
    BLOCK_FEATURES,
    BLOCK_FEATURES_NEIGHBORHOOD,
    STREET_FEATURES,
    STREET_FEATURES_NEIGHBORHOOD,
    STREET_FEATURES_CENTRALITY,
    CITY_FEATURES,
    # LANDUSE_FEATURES,
))

## Spatial utils

Functions for processing the geographic coordinates.

In [7]:
from scipy.stats import wasserstein_distance
from math import radians, cos, sin, asin, sqrt

def deg_to_rad(x):
  return x * math.pi / 180

def latlon_to_cart(lat,lon):
  x = np.cos(lat) * np.cos(lon)
  y = np.cos(lat) * np.sin(lon)
  z = np.sin(lat)
  cart_coord = np.column_stack((x, y, z))
  return cart_coord

def haversine(lon1, lat1, lon2, lat2): 
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 
    return c * r * 1000

# Helper function for 2+d distance
def newDistance(a, b, nd_dist="great_circle"):  
    # Distance options are ["great_circle" (2D only), "euclidean", "wasserstein" (for higher-dimensional coordinate embeddings)]
    if a.shape[0]==2:
      x1, y1 = a[0], a[1]
      x2, y2 = b[0], b[1]
      if nd_dist=="euclidean":
        d = math.sqrt( ((x1-x2)**2)+((y1-y2)**2))
      else:
        d = haversine(x1,y1,x2,y2)
    if a.shape[0]==3:
      x1, y1, z1 = a[0], a[1], a[2]
      x2, y2, z2 = b[0], b[1], b[2]
      d = math.sqrt(math.pow(x2 - x1, 2) +
                  math.pow(y2 - y1, 2) +
                  math.pow(z2 - z1, 2)* 1.0) 
    if a.shape[0]>3:
      if nd_dist=="wasserstein":
        d = wasserstein_distance(a.reshape(-1).detach(),b.reshape(-1).detach())
        #d = sgw_cpu(a.reshape(1,-1).detach(),b.reshape(1,-1).detach())
      else:
        d = torch.pow(a.reshape(1,1,-1) - b.reshape(1,1,-1), 2).sum(2) 
    return d 

# Helper function for edge weights
def makeEdgeWeight(x, edge_index):
  to = edge_index[0]
  fro = edge_index[1]
  edge_weight = []
  for i in range(len(to)):
    edge_weight.append(newDistance(x[to[i]],x[fro[i]])) # probably want to do inverse distance eventually
  max_val = max(edge_weight)
  rng = max_val - min(edge_weight)
  edge_weight = [(max_val - elem) / rng for elem in edge_weight]
  return torch.Tensor(edge_weight)

# knn graph to adjacency matrix (probably already built)
def knn_to_adj(knn, n):
  adj_matrix = torch.zeros(n, n, dtype=float) #lil_matrix((n, n), dtype=float) 
  for i in range(len(knn[0])):
    tow = knn[0][i]
    fro = knn[1][i]
    adj_matrix[tow,fro] = 1 # should be bidectional?
  return adj_matrix.T

def normal_torch(tensor,min_val=0):
  t_min = torch.min(tensor)
  t_max = torch.max(tensor)
  if t_min == 0 and t_max == 0:
    return torch.tensor(tensor)
  if min_val == -1:
    tensor_norm = 2 * ((tensor - t_min) / (t_max - t_min)) - 1
  if min_val== 0:
    tensor_norm = ((tensor - t_min) / (t_max - t_min))
  return torch.tensor(tensor_norm)

def lw_tensor_local_moran(y,w_sparse,na_to_zero=True,norm=True,norm_min_val=0):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  y = y.reshape(-1)
  n = len(y)
  n_1 = n - 1
  z = y - y.mean()
  sy = y.std()
  z /= sy
  den = (z * z).sum()
  zl = torch.tensor(w_sparse * z).to(device)
  mi = n_1 * z * zl / den
  if na_to_zero==True:
    mi[torch.isnan(mi)] = 0
  if norm==True:
    mi = normal_torch(mi,min_val=norm_min_val)
  return torch.tensor(mi)

## Model

Here are the different modules needed for PE-GNN. The Positional Encoder (PE) modules are adapted building on the original [Space2Vec](https://openreview.net/forum?id=rJljdh4KDH) code: https://github.com/gengchenmai/space2vec.

In [8]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F

import torch.utils.data
import math

class LayerNorm(nn.Module):
    """
    layer normalization
    Simple layer norm object optionally used with the convolutional encoder.
    """

    def __init__(self, feature_dim, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones((feature_dim,)))
        self.register_parameter("gamma", self.gamma)
        self.beta = nn.Parameter(torch.zeros((feature_dim,)))
        self.register_parameter("beta", self.beta)
        self.eps = eps

    def forward(self, x):
        # x: [batch_size, embed_dim]
        # normalize for each embedding
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        # output shape is the same as x
        # Type not match for self.gamma and self.beta??????????????????????
        # output: [batch_size, embed_dim]
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

def get_activation_function(activation, context_str):
    if activation == "leakyrelu":
        return nn.LeakyReLU(negative_slope=0.2)
    elif activation == "relu":
        return nn.ReLU()
    elif activation == "sigmoid":
        return nn.Sigmoid()
    elif activation == 'tanh':
        return nn.Tanh()
    else:
        raise Exception("{} activation not recognized.".format(context_str))


class SingleFeedForwardNN(nn.Module):
    """
        Creates a single layer fully connected feed forward neural network.
        this will use non-linearity, layer normalization, dropout
        this is for the hidden layer, not the last layer of the feed forard NN
    """

    def __init__(self, input_dim,
                    output_dim,
                    dropout_rate=None,
                    activation="sigmoid",
                    use_layernormalize=False,
                    skip_connection = False,
                    context_str = ''):
        '''
        Args:
            input_dim (int32): the input embedding dim
            output_dim (int32): dimension of the output of the network.
            dropout_rate (scalar tensor or float): Dropout keep prob.
            activation (string): tanh or relu or leakyrelu or sigmoid
            use_layernormalize (bool): do layer normalization or not
            skip_connection (bool): do skip connection or not
            context_str (string): indicate which spatial relation encoder is using the current FFN
        '''
        super(SingleFeedForwardNN, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

        if dropout_rate is not None:
            self.dropout = nn.Dropout(p=dropout_rate)
        else:
            self.dropout = None

        self.act = get_activation_function(activation, context_str)

        if use_layernormalize:
            # the layer normalization is only used in the hidden layer, not the last layer
            self.layernorm = nn.LayerNorm(self.output_dim)
        else:
            self.layernorm = None

        # the skip connection is only possible, if the input and out dimention is the same
        if self.input_dim == self.output_dim:
            self.skip_connection = skip_connection
        else:
            self.skip_connection = False
        
        self.linear = nn.Linear(self.input_dim, self.output_dim)
        nn.init.xavier_uniform(self.linear.weight)
        




    def forward(self, input_tensor):
        '''
        Args:
            input_tensor: shape [batch_size, ..., input_dim]
        Returns:
            tensor of shape [batch_size,..., output_dim]
            note there is no non-linearity applied to the output.
        Raises:
            Exception: If given activation or normalizer not supported.
        '''
        assert input_tensor.size()[-1] == self.input_dim
        # Linear layer
        output = self.linear(input_tensor)
        # non-linearity
        output = self.act(output)
        # dropout
        if self.dropout is not None:
            output = self.dropout(output)

        # skip connection
        if self.skip_connection:
            output = output + input_tensor

        # layer normalization
        if self.layernorm is not None:
            output = self.layernorm(output)

        return output

class MultiLayerFeedForwardNN(nn.Module):
    """
        Creates a fully connected feed forward neural network.
        N fully connected feed forward NN, each hidden layer will use non-linearity, layer normalization, dropout
        The last layer do not have any of these
    """

    def __init__(self, input_dim,
                    output_dim,
                    num_hidden_layers=0,
                    dropout_rate=0.5,
                    hidden_dim=-1,
                    activation="relu",
                    use_layernormalize=True,
                    skip_connection = False,
                    context_str = None):
        '''
        Args:
            input_dim (int32): the input embedding dim
            num_hidden_layers (int32): number of hidden layers in the network, set to 0 for a linear network.
            output_dim (int32): dimension of the output of the network.
            dropout (scalar tensor or float): Dropout keep prob.
            hidden_dim (int32): size of the hidden layers
            activation (string): tanh or relu
            use_layernormalize (bool): do layer normalization or not
            context_str (string): indicate which spatial relation encoder is using the current FFN
        '''
        super(MultiLayerFeedForwardNN, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_hidden_layers = num_hidden_layers
        self.dropout_rate = dropout_rate
        self.hidden_dim = hidden_dim
        self.activation = activation
        self.use_layernormalize = use_layernormalize
        self.skip_connection = skip_connection
        self.context_str = context_str

        self.layers = nn.ModuleList()
        if self.num_hidden_layers <= 0:
            self.layers.append( SingleFeedForwardNN(input_dim = self.input_dim,
                                                    output_dim = self.output_dim,
                                                    dropout_rate = self.dropout_rate,
                                                    activation = self.activation,
                                                    use_layernormalize = False,
                                                    skip_connection = False,
                                                    context_str = self.context_str))
        else:
            self.layers.append( SingleFeedForwardNN(input_dim = self.input_dim,
                                                    output_dim = self.hidden_dim,
                                                    dropout_rate = self.dropout_rate,
                                                    activation = self.activation,
                                                    use_layernormalize = self.use_layernormalize,
                                                    skip_connection = self.skip_connection,
                                                    context_str = self.context_str))

            for i in range(self.num_hidden_layers-1):
                self.layers.append( SingleFeedForwardNN(input_dim = self.hidden_dim,
                                                    output_dim = self.hidden_dim,
                                                    dropout_rate = self.dropout_rate,
                                                    activation = self.activation,
                                                    use_layernormalize = self.use_layernormalize,
                                                    skip_connection = self.skip_connection,
                                                    context_str = self.context_str))

            self.layers.append( SingleFeedForwardNN(input_dim = self.hidden_dim,
                                                    output_dim = self.output_dim,
                                                    dropout_rate = self.dropout_rate,
                                                    activation = self.activation,
                                                    use_layernormalize = False,
                                                    skip_connection = False,
                                                    context_str = self.context_str))

        

    def forward(self, input_tensor):
        '''
        Args:
            input_tensor: shape [batch_size, ..., input_dim]
        Returns:
            tensor of shape [batch_size, ..., output_dim]
            note there is no non-linearity applied to the output.
        Raises:
            Exception: If given activation or normalizer not supported.
        '''
        assert input_tensor.size()[-1] == self.input_dim
        output = input_tensor
        for i in range(len(self.layers)):
            output = self.layers[i](output)

        return output

def _cal_freq_list(freq_init, frequency_num, max_radius, min_radius):
    if freq_init == "random":
        freq_list = np.random.random(size=[frequency_num]) * max_radius
    elif freq_init == "geometric":
        log_timescale_increment = (math.log(float(max_radius) / float(min_radius)) / (frequency_num*1.0 - 1))
        timescales = min_radius * np.exp(np.arange(frequency_num).astype(float) * log_timescale_increment)
        freq_list = 1.0/timescales
    return freq_list

class GridCellSpatialRelationEncoder(nn.Module):
    """
    Given a list of (deltaX,deltaY), encode them using the position encoding function
    """
    def __init__(self, spa_embed_dim, coord_dim = 2, frequency_num = 16, 
            max_radius =0.01, min_radius = 0.00001,
            freq_init = "geometric",
            ffn=None):
        """
        Args:
            spa_embed_dim: the output spatial relation embedding dimention
            coord_dim: the dimention of space, 2D, 3D, or other
            frequency_num: the number of different sinusoidal with different frequencies/wavelengths
            max_radius: the largest context radius this model can handle
        """
        super(GridCellSpatialRelationEncoder, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.spa_embed_dim = spa_embed_dim
        self.coord_dim = coord_dim 
        self.frequency_num = frequency_num
        self.freq_init = freq_init
        self.max_radius = max_radius
        self.min_radius = min_radius
        self.ffn = ffn
        # the frequence we use for each block, alpha in ICLR paper
        self.cal_freq_list()
        self.cal_freq_mat()
        self.input_embed_dim = self.cal_input_dim()

        if self.ffn is not None:
          self.ffn = MultiLayerFeedForwardNN(2 * frequency_num * 2, spa_embed_dim)

    def cal_elementwise_angle(self, coord, cur_freq):
        '''
        Args:
            coord: the deltaX or deltaY
            cur_freq: the frequency
        '''
        return coord/(np.power(self.max_radius, cur_freq*1.0/(self.frequency_num-1)))

    def cal_coord_embed(self, coords_tuple):
        embed = []
        for coord in coords_tuple:
            for cur_freq in range(self.frequency_num):
                embed.append(math.sin(self.cal_elementwise_angle(coord, cur_freq)))
                embed.append(math.cos(self.cal_elementwise_angle(coord, cur_freq)))
        # embed: shape (input_embed_dim)
        return embed

    def cal_input_dim(self):
        # compute the dimention of the encoded spatial relation embedding
        return int(self.coord_dim * self.frequency_num * 2)

    def cal_freq_list(self):
        self.freq_list = _cal_freq_list(self.freq_init, self.frequency_num, self.max_radius, self.min_radius)


    def cal_freq_mat(self):
        # freq_mat shape: (frequency_num, 1)
        freq_mat = np.expand_dims(self.freq_list, axis = 1)
        # self.freq_mat shape: (frequency_num, 2)
        self.freq_mat = np.repeat(freq_mat, 2, axis = 1)

    def make_input_embeds(self, coords):
        if type(coords) == np.ndarray:
            assert self.coord_dim == np.shape(coords)[2]
            coords = list(coords)
        elif type(coords) == list:
            assert self.coord_dim == len(coords[0][0])
        else:
            raise Exception("Unknown coords data type for GridCellSpatialRelationEncoder")
        
        # coords_mat: shape (batch_size, num_context_pt, 2)
        coords_mat = np.asarray(coords).astype(float)
        batch_size = coords_mat.shape[0]
        num_context_pt = coords_mat.shape[1]
        # coords_mat: shape (batch_size, num_context_pt, 2, 1)
        coords_mat = np.expand_dims(coords_mat, axis = 3)
        # coords_mat: shape (batch_size, num_context_pt, 2, 1, 1)
        coords_mat = np.expand_dims(coords_mat, axis = 4)
        # coords_mat: shape (batch_size, num_context_pt, 2, frequency_num, 1)
        coords_mat = np.repeat(coords_mat, self.frequency_num, axis = 3)
        # coords_mat: shape (batch_size, num_context_pt, 2, frequency_num, 2)
        coords_mat = np.repeat(coords_mat, 2, axis = 4)
        # spr_embeds: shape (batch_size, num_context_pt, 2, frequency_num, 2)
        spr_embeds = coords_mat * self.freq_mat
        # make sinuniod function
        # sin for 2i, cos for 2i+1
        # spr_embeds: (batch_size, num_context_pt, 2*frequency_num*2=input_embed_dim)
        spr_embeds[:, :, :, :, 0::2] = np.sin(spr_embeds[:, :, :, :, 0::2])  # dim 2i
        spr_embeds[:, :, :, :, 1::2] = np.cos(spr_embeds[:, :, :, :, 1::2])  # dim 2i+1
        # (batch_size, num_context_pt, 2*frequency_num*2)
        spr_embeds = np.reshape(spr_embeds, (batch_size, num_context_pt, -1))
        return spr_embeds

    def forward(self, coords):
        """
        Given a list of coords (deltaX, deltaY), give their spatial relation embedding
        Args:
            coords: a python list with shape (batch_size, num_context_pt, coord_dim)
        Return:
            sprenc: Tensor shape (batch_size, num_context_pt, spa_embed_dim)
        """   
        spr_embeds = self.make_input_embeds(coords)
        spr_embeds = torch.FloatTensor(spr_embeds).to(self.device)
        if self.ffn is not None:
            return self.ffn(spr_embeds)
        else:
            return spr_embeds

The GNN backbones are defined here, with a GCN as example (though other layers like GAT or GraphSAGE can be used interchangeably with the GCN layers), using *PyTorch Geometric*.

In [9]:
class GCN(nn.Module):
    """
        GCN
    """
    def __init__(self, num_features_in=3, num_features_out=1, k=20, MAT=False):
        super(GCN, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.k = k
        self.MAT = MAT
        self.conv1 = GCNConv(num_features_in, 32)
        self.conv2 = GCNConv(32, 32)
        self.fc = nn.Linear(32, num_features_out)
        if MAT:
          self.fc_morans = nn.Linear(32, num_features_out)
    def forward(self, x, c, ei, ew):
        x = x.float()
        c = c.float()
        if torch.is_tensor(ei) & torch.is_tensor(ew):
          edge_index = ei
          edge_weight = ew
        else:
          edge_index = knn_graph(c, k=self.k).to(self.device)
          edge_weight = makeEdgeWeight(c, edge_index).to(self.device)
        h1 = F.relu(self.conv1(x, edge_index, edge_weight))
        h1 = F.dropout(h1, training=self.training)
        h2 = F.relu(self.conv2(h1, edge_index, edge_weight))
        h2 = F.dropout(h2, training=self.training)
        output = self.fc(h2)
        if self.MAT:
          morans_output = self.fc_morans(h2)
          return output, morans_output
        else:
          return output

class PEGCN(nn.Module):
    """
        GCN with positional encoder and auxiliary tasks
    """
    def __init__(self, num_features_in=3, num_features_out=1, emb_hidden_dim=128, emb_dim=16, k = 20, MAT=False):
        super(PEGCN, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.emb_hidden_dim = emb_hidden_dim
        self.emb_dim = emb_dim
        self.k = k
        self.MAT = MAT
        self.spenc = GridCellSpatialRelationEncoder(spa_embed_dim=emb_hidden_dim,ffn=True,min_radius=1e-06,max_radius=360)
        self.dec = nn.Sequential(
            nn.Linear(emb_hidden_dim, emb_hidden_dim // 2),
            nn.Tanh(),
            nn.Linear(emb_hidden_dim // 2, emb_hidden_dim // 4),
            nn.Tanh(),
            nn.Linear(emb_hidden_dim // 4, emb_dim)
        )
        self.conv1 = GCNConv(num_features_in + emb_dim, 32)
        self.conv2 = GCNConv(32, 32)
        self.fc = nn.Linear(32, num_features_out)
        if MAT:
          self.fc_morans = nn.Linear(32, num_features_out)
    def forward(self, x, c, ei, ew):
        x = x.float()
        c = c.float()
        if torch.is_tensor(ei) & torch.is_tensor(ew):
          edge_index = ei
          edge_weight = ew
        else:
          edge_index = knn_graph(c, k=self.k).to(self.device)
          edge_weight = makeEdgeWeight(c, edge_index).to(self.device)

        c = c.reshape(1, c.shape[0], c.shape[1])
        emb = self.spenc(c.detach().cpu().numpy())
        emb = emb.reshape(emb.shape[1],emb.shape[2])
        emb = self.dec(emb).float()
        x = torch.cat((x,emb),dim=1)

        h1 = F.relu(self.conv1(x, edge_index, edge_weight))
        h1 = F.dropout(h1, training=self.training)
        h2 = F.relu(self.conv2(h1, edge_index, edge_weight))
        h2 = F.dropout(h2, training=self.training)
        output = self.fc(h2)
        if self.MAT:
          morans_output = self.fc_morans(h2)
          return output, morans_output
        else:
          return output

class LossWrapper(nn.Module):
    def __init__(self, model, task_num=1, loss='mse', uw=True, lamb=0.5, k=20, batch_size=2048):
        super(LossWrapper, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model
        self.task_num = task_num
        self.uw = uw
        self.lamb = lamb
        self.k = k
        self.batch_size = batch_size
        if task_num > 1:
          self.log_vars = nn.Parameter(torch.zeros((task_num)))
        if loss=="mse":
          self.criterion = nn.MSELoss()
        elif loss=="l1":
          self.criterion = nn.L1Loss()

    def forward(self, input, targets, coords, edge_index, edge_weight, morans_input):

        if self.task_num==1:
          outputs = self.model(input, coords, edge_index, edge_weight)
          loss = self.criterion(targets.float().reshape(-1),outputs.float().reshape(-1))
          return loss

        else:
          outputs1, outputs2 = self.model(input, coords, edge_index, edge_weight)
          if torch.is_tensor(morans_input):
            targets2 = morans_input
          else:
            moran_weight_matrix = knn_to_adj(knn_graph(coords, k=self.k), self.batch_size) 
            with torch.enable_grad():
              targets2 = lw_tensor_local_moran(targets, sparse.csr_matrix(moran_weight_matrix)).to(self.device)
          if self.uw:
            precision1 = 0.5 * torch.exp(-self.log_vars[0])
            loss1 = self.criterion(targets.float().reshape(-1),outputs1.float().reshape(-1))
            loss1 = torch.sum(precision1 * loss1 + self.log_vars[0], -1)

            precision2 = 0.5 * torch.exp(-self.log_vars[1])
            loss2 = self.criterion(targets2.float().reshape(-1),outputs2.float().reshape(-1))
            loss2 = torch.sum(precision2 * loss2 + self.log_vars[1], -1)

            loss = loss1 + loss2
            loss = torch.mean(loss)
            return loss, self.log_vars.data.tolist()
          else:
            loss1 = self.criterion(targets.float().reshape(-1),outputs1.float().reshape(-1))
            loss2 = self.criterion(targets2.float().reshape(-1),outputs2.float().reshape(-1))
            loss = loss1 + self.lamb * loss2
            return loss        

## Training

The final training loop for *PE-GNN*.

In [10]:
model = None

def train(args):
  # Get args
  dset = args.dset
  model_name = args.model_name
  random_state = args.random_state
  path = args.path
  train_size = args.train_size
  batched_training = args.batched_training
  batch_size = args.batch_size
  n_epochs = args.n_epochs
  train_crit = args.train_crit
  lr = args.lr
  emb_dim = args.emb_dim
  MAT = args.mat
  uw = args.uw
  lamb = args.lamb
  k = args.k
  save_freq = args.save_freq
  print_progress = args.print_progress

  # Set random seed
  np.random.seed(random_state)

  # Set device
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # Access and process data
  if dset=="cali_housing":
    c,x,y = get_cali_housing_data()
  if dset=="election":
    c,x,y = get_election_data()
  if dset=="air_temp":
    c,x,y = get_air_temp_data()
  if dset=="3d_road":
    c,x,y = get_3d_road_data()
    x = torch.ones(y.shape[0],1) #Dummies
  if dset=="eubucco":
    c,x,y = get_eubucco_data()

  n = x.shape[0]
  n_train = np.round(n * train_size).astype(int)
  n_test = (n - n_train).astype(int)
  indices = np.arange(n)
  _, _, _, _, idx_train, idx_test = train_test_split(x, y, indices, test_size=(1-train_size), random_state=random_state)

  train_x, test_x = x[idx_train], x[idx_test]
  train_y, test_y = y[idx_train], y[idx_test]
  train_c, test_c = c[idx_train], c[idx_test]
  train_dataset, test_dataset = MyDataset(train_x, train_y, train_c), MyDataset(test_x, test_y, test_c)
  
  if batched_training==False:
    batch_size = len(idx_train)
    train_edge_index = knn_graph(train_c, k=k).to(device)
    train_edge_weight = makeEdgeWeight(train_c, train_edge_index).to(device)
    test_edge_index = knn_graph(test_c, k=k).to(device)
    test_edge_weight = makeEdgeWeight(test_c, test_edge_index).to(device)
    train_moran_weight_matrix = knn_to_adj(train_edge_index, batch_size) #libpysal.weights.KNN(batch_y.cpu(), k=20).to(device)
    with torch.enable_grad():
      train_y_moran = lw_tensor_local_moran(train_y, sparse.csr_matrix(train_moran_weight_matrix)).to(device)
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=False, drop_last=False)
  else:
    train_edge_index = False
    train_edge_weight = False
    test_edge_index = False
    test_edge_weight = False
    train_y_moran = False
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, drop_last=True)

  # Make model
  if model_name=="gcn":
    model = GCN(num_features_in=train_x.shape[1],k=k,MAT=MAT).to(device)
  if model_name=="pegcn":
    model = PEGCN(num_features_in=train_x.shape[1],k=k,MAT=MAT,emb_dim=emb_dim).to(device)
  model = model.float()

  if MAT:
    task_num = 2
  else:
    task_num = 1

  loss_wrapper = LossWrapper(model, task_num=task_num, loss=train_crit, uw=uw, lamb=lamb, k=k, batch_size=batch_size).to(device)
  optimizer = torch.optim.Adam(loss_wrapper.parameters(), lr=lr)
  score1 = nn.MSELoss()
  score2 = nn.L1Loss()
  
  # Tensorboard and logging
  test_ = dset + '-' + model_name + '-k' + str(k)
  if model_name=='pegcn':
    test_ = test_ + '-emb' + str(emb_dim)
  if MAT:
    if uw:
      test_ = test_ + "_mat-uw"
    else:
      test_ = test_ + "_mat-lam" + str(lamb)
  if batched_training==True:
    test_ = test_ + "_bs" + str(batch_size) + "_ep" + str(n_epochs)
  else:
    test_ = test_ + "_bsn_ep" + str(n_epochs)

  saved_file = "{}_{}{}-{}:{}:{}.{}".format(test_,
                                            datetime.now().strftime("%h"),
                                            datetime.now().strftime("%d"),
                                            datetime.now().strftime("%H"),
                                            datetime.now().strftime("%M"),
                                            datetime.now().strftime("%S"),
                                            datetime.now().strftime("%f"))

  log_dir = path + "/trained/{}/log".format(saved_file)

  if not os.path.exists(path + "trained/{}/data".format(saved_file)):
      os.makedirs(path + "/trained/{}/data".format(saved_file))
  if not os.path.exists(path + "/trained/{}/images".format(saved_file)):
      os.makedirs(path + "/trained/{}/images".format(saved_file))
  with open(path + "/trained/{}/train_notes.txt".format(saved_file), 'w') as f:
      # Include any experiment notes here:
      f.write("Experiment notes: .... \n\n")
      f.write("MODEL_DATA: {}\n".format(
          test_))
      f.write("BATCH_SIZE: {}\nLEARNING_RATE: {}\n".format(
          batch_size,
          lr))
  
  writer = SummaryWriter(log_dir)
  
  # Training loop
  it_counts = 0
  for epoch in range(n_epochs):
    for batch in train_loader:
      model.train()
      it_counts += 1
      x = batch[0].to(device).float()
      y = batch[1].to(device).float()
      c = batch[2].to(device).float()

      optimizer.zero_grad()

      if MAT==True & uw==True:
        loss, log_vars = loss_wrapper(x, y, c, train_edge_index, train_edge_weight, train_y_moran)
      else:
        loss = loss_wrapper(x, y, c, train_edge_index, train_edge_weight, train_y_moran)
      loss.backward()
      optimizer.step()
      # Eval 
      if it_counts % save_freq == 0:
        model.eval()
        with torch.no_grad():
          if MAT:
            pred,_ = model(torch.tensor(test_dataset.features).to(device),torch.tensor(test_dataset.coords).to(device),test_edge_index,test_edge_weight)
          else:
            pred = model(torch.tensor(test_dataset.features).to(device),torch.tensor(test_dataset.coords).to(device),test_edge_index,test_edge_weight)
        test_score1 = score1(torch.tensor(test_dataset.target).reshape(-1).to(device), pred.reshape(-1))
        test_score2 = score2(torch.tensor(test_dataset.target).reshape(-1).to(device), pred.reshape(-1))

        if print_progress:
          print("Epoch [%d/%d] - Loss: %f - Test score (MSE): %f - Test score (MAE): %f" % (epoch, n_epochs, loss.item(), test_score1.item(), test_score2.item()))
        save_path = path + "/trained/{}/ckpts".format(saved_file)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        torch.save(model, save_path + '/' + 'model.pt')
        if test_score2 < 50:
          torch.save(model, save_path + '/' + 'model-50.pt')
        writer.add_scalar('Test score (MSE)', test_score1.item(), it_counts)
        writer.add_scalar('Test score (MAE)', test_score2.item(), it_counts)
      writer.add_scalar('Training loss', loss.item(), it_counts)
      if MAT==True & uw==True:
        writer.add_scalar('Uncertainty weight: main task', log_vars[0], it_counts)
        writer.add_scalar('Uncertainty weight: Morans aux task', log_vars[1], it_counts)  
      writer.flush()
  print("Saved all models to {}".format(save_path))
  print("Epoch [%d/%d] - Loss: %f - Test score (MSE): %f - Test score (MAE): %f" % (epoch, n_epochs, loss.item(), test_score1.item(), test_score2.item()))

Load tensorboard

In [11]:
%tensorboard --logdir=trained

UsageError: Line magic function `%tensorboard` not found.


## Training 

Set your parameters and train away!

In [12]:
class Args:
    dset = 'eubucco'
    model_name = 'pegcn'
    random_state = 1
    path = './'
    train_size = 0.8
    batched_training = True
    batch_size = 1024
    n_epochs = 50
    train_crit = 'mse'
    lr = 1e-3
    emb_dim = 64
    mat = False
    uw = False
    lamb = 0.5
    k = 5
    save_freq = 5
    print_progress = True

In [20]:
train(Args)

/var/folders/jk/rndbblbs2fs0_symsqry30400000gn/T/ipykernel_4105/2829827725.py:94: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.linear.weight)
/var/folders/jk/rndbblbs2fs0_symsqry30400000gn/T/ipykernel_4105/1093280713.py:222: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.features[idx]), torch.tensor(self.target[idx]), torch.tensor(self.coords[idx])
/var/folders/jk/rndbblbs2fs0_symsqry30400000gn/T/ipykernel_4105/145656972.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = model(torch.tensor(test_dataset.features).to(device),torch.tensor(test_dataset.coords).to(device),test_edge_in

Epoch [0/50] - Loss: 3901880.500000 - Test score (MSE): 3901880.472470 - Test score (MAE): 1975.159423
Epoch [0/50] - Loss: 3892301.000000 - Test score (MSE): 3899003.129780 - Test score (MAE): 1974.430893
Epoch [0/50] - Loss: 3897534.000000 - Test score (MSE): 3893876.076935 - Test score (MAE): 1973.132080
Epoch [0/50] - Loss: 3888918.000000 - Test score (MSE): 3886084.610257 - Test score (MAE): 1971.156688
Epoch [0/50] - Loss: 3884762.250000 - Test score (MSE): 3875194.139056 - Test score (MAE): 1968.392300
Epoch [0/50] - Loss: 3867599.250000 - Test score (MSE): 3860085.074438 - Test score (MAE): 1964.550631
Epoch [0/50] - Loss: 3841316.000000 - Test score (MSE): 3839491.682686 - Test score (MAE): 1959.302363
Epoch [0/50] - Loss: 3815416.250000 - Test score (MSE): 3811605.019115 - Test score (MAE): 1952.172897
Epoch [0/50] - Loss: 3783448.250000 - Test score (MSE): 3774484.322772 - Test score (MAE): 1942.642066
Epoch [0/50] - Loss: 3741243.500000 - Test score (MSE): 3725259.607277 - 

In [14]:
pretrained_model = torch.load('trained/eubucco-pegcn-k5-emb64_bs1024_ep50_Nov22-10:16:53.274287/ckpts/model-50.pt')
pretrained_model

PEGCN(
  (spenc): GridCellSpatialRelationEncoder(
    (ffn): MultiLayerFeedForwardNN(
      (layers): ModuleList(
        (0): SingleFeedForwardNN(
          (dropout): Dropout(p=0.5, inplace=False)
          (act): ReLU()
          (linear): Linear(in_features=64, out_features=128, bias=True)
        )
      )
    )
  )
  (dec): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Tanh()
    (4): Linear(in_features=32, out_features=64, bias=True)
  )
  (conv1): GCNConv(90, 32)
  (conv2): GCNConv(32, 32)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

In [15]:
# save lat, lon embeddings

c, _, _ = get_eubucco_data()

# for c in coords:
print(c)
print(c.shape)
c = c.reshape(1, c.shape[0], c.shape[1])
emb = pretrained_model.spenc(c.detach().cpu().numpy())
emb = emb.reshape(emb.shape[1],emb.shape[2])
print(emb)
print(emb.shape)

tensor([[52.1889,  5.9965],
        [52.1882,  6.0017],
        [52.1885,  5.9982],
        ...,
        [51.2725,  5.9548],
        [51.2729,  5.9651],
        [51.2902,  5.9058]], dtype=torch.float64)
torch.Size([174210, 2])
tensor([[0.0000, 0.2507, 0.4268,  ..., 0.5736, 0.2886, 1.3723],
        [0.0000, 0.4717, 1.7743,  ..., 1.0364, 0.0000, 2.2306],
        [0.0000, 0.6606, 1.5635,  ..., 1.0993, 0.7683, 1.6891],
        ...,
        [0.0000, 1.5621, 1.1701,  ..., 0.3243, 0.0819, 1.9081],
        [0.0000, 0.6688, 1.3038,  ..., 0.2738, 0.4300, 1.7834],
        [0.0000, 0.7901, 0.3934,  ..., 0.0639, 1.3535, 2.7593]],
       grad_fn=<ReshapeAliasBackward0>)
torch.Size([174210, 128])
